In [1]:
import pandas as pd
churn_data = pd.read_csv('data/churn.csv')
churn_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### <center>9.1. Каково соотношение ушедших и лояльных клиентов?

In [3]:
import plotly
import plotly.express as px


### <center>9.2. Постройте график, показывающий распределение баланса пользователей, у которых на счету больше 2 500 долларов.

In [ ]:
import plotly
import plotly.express as px

### <center>9.3. Посмотрите на распределение баланса клиента в разрезе признака оттока. Как различаются суммы на накопительном счёте ушедших и лояльных клиентов?

In [ ]:
import plotly
import plotly.express as px

### <center>9.4. Посмотрите на распределение возраста в разрезе признака оттока. В какой группе больше потенциальных выбросов? На какую возрастную категорию клиентов стоит обратить внимание банку?

In [ ]:
import plotly
import plotly.express as px

### <center>9.5. Постройте график, который показывает взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты. Добавьте расцветку по признаку оттока клиентов. Какова взаимосвязь между признаками?

In [ ]:
import plotly
import plotly.express as px

### <center>9.6. Кто чаще уходит, мужчины или женщины? Постройте график, который иллюстрирует это.

In [ ]:
import plotly
import plotly.express as px

### <center>9.7. Как отток клиентов зависит от числа приобретённых у банка услуг?

In [ ]:
import plotly
import plotly.express as px

### <center>9.8. Как влияет наличие статуса активного клиента на отток клиентов? Постройте диаграмму, иллюстрирующую это. Что бы вы предложили банку, чтобы уменьшить отток клиентов среди неактивных?

In [ ]:
import plotly
import plotly.express as px

### <center>9.9. В какой стране доля ушедших клиентов больше? Постройте тепловую картограмму, которая покажет это соотношение на карте мира.

In [ ]:
import plotly
import plotly.express as px

### <center>9.10.Найдите на тепловой карте категории клиентов, которые уходят чаще всего.

In [ ]:
import plotly
import plotly.express as px